## Het trainen van een CBoW-model

Deze notebook maakt deel uit van [AI for Beginners Curriculum](http://aka.ms/ai-beginners)

In dit voorbeeld gaan we kijken naar het trainen van een CBoW-taalmodel om onze eigen Word2Vec-embeddingruimte te verkrijgen. We zullen de AG News-dataset gebruiken als bron van tekst.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Laten we eerst onze dataset laden en de tokenizer en woordenschat definiëren. We stellen `vocab_size` in op 5000 om de berekeningen enigszins te beperken.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW-model

CBoW leert een woord te voorspellen op basis van de $2N$ omliggende woorden. Bijvoorbeeld, wanneer $N=1$, krijgen we de volgende paren uit de zin *I like to train networks*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Hier is het eerste woord het omliggende woord dat als invoer wordt gebruikt, en het tweede woord is het woord dat we voorspellen.

Om een netwerk te bouwen dat het volgende woord voorspelt, moeten we het omliggende woord als invoer geven en het woordnummer als uitvoer krijgen. De architectuur van het CBoW-netwerk is als volgt:

* Het invoerwoord wordt door de embedding-laag geleid. Deze embedding-laag wordt onze Word2Vec-embedding, dus we definiëren deze apart als de variabele `embedder`. In dit voorbeeld gebruiken we een embedding-grootte van 30, hoewel je wellicht wilt experimenteren met hogere dimensies (echte Word2Vec heeft 300).
* De embedding-vector wordt vervolgens doorgegeven aan een lineaire laag die het uitvoerwoord voorspelt. Deze laag heeft dus `vocab_size` neuronen.

Voor de uitvoer, als we `CrossEntropyLoss` als verliesfunctie gebruiken, moeten we alleen woordnummers als verwachte resultaten aanleveren, zonder one-hot encoding.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Trainingsgegevens voorbereiden

Laten we nu de hoofdfunctie programmeren die CBoW-woordparen uit tekst zal berekenen. Met deze functie kunnen we de venstergrootte specificeren en zal deze een set paren retourneren - invoer- en uitvoerwoord. Let op dat deze functie zowel op woorden als op vectoren/tensors kan worden gebruikt - wat ons in staat stelt de tekst te coderen voordat we deze doorgeven aan de functie `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Laten we de trainingsdataset voorbereiden. We zullen alle nieuwsitems doorlopen, `to_cbow` aanroepen om de lijst met woordparen te verkrijgen, en die paren toevoegen aan `X` en `Y`. Om tijd te besparen, zullen we alleen de eerste 10k nieuwsitems beschouwen - je kunt de beperking eenvoudig verwijderen als je meer tijd hebt om te wachten en betere embeddings wilt krijgen :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

We zullen die gegevens ook omzetten naar één dataset en een dataloader maken:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

We zullen die gegevens ook omzetten naar één dataset en een dataloader maken:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Laten we nu beginnen met de daadwerkelijke training. We zullen de `SGD` optimizer gebruiken met een vrij hoge leersnelheid. Je kunt ook experimenteren met andere optimizers, zoals `Adam`. We zullen beginnen met trainen voor 10 epochs - en je kunt deze cel opnieuw uitvoeren als je een nog lagere verlieswaarde wilt.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec uitproberen

Om Word2Vec te gebruiken, laten we vectoren extraheren die overeenkomen met alle woorden in onze woordenschat:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Laten we eens kijken, bijvoorbeeld, hoe het woord **Parijs** wordt gecodeerd in een vector:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Het is interessant om Word2Vec te gebruiken om naar synoniemen te zoeken. De volgende functie zal `n` dichtstbijzijnde woorden retourneren voor een gegeven invoer. Om ze te vinden, berekenen we de norm van $|w_i - v|$, waarbij $v$ de vector is die overeenkomt met ons invoerwoord, en $w_i$ de codering is van het $i$-de woord in de vocabulaire. Vervolgens sorteren we de array en retourneren de overeenkomstige indices met behulp van `argsort`, en nemen de eerste `n` elementen van de lijst, die de posities van de dichtstbijzijnde woorden in de vocabulaire coderen.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Belangrijkste punten

Met slimme technieken zoals CBoW kunnen we een Word2Vec-model trainen. Je kunt ook proberen een skip-gram model te trainen, dat wordt getraind om het naburige woord te voorspellen gegeven het centrale woord, en kijken hoe goed het presteert.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
